# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd

#file path
tamil_dataset = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'


# Step 3: Load the Dataset
# Assuming it's a CSV file
dataset = pd.read_csv(tamil_dataset)


# Step 4: Display the Dataset
# Show the first few rows
print(dataset.head())

print()
# Show information about the dataset
print(dataset.info())


                ID                                               DATA  LABEL
0  MAL_HUAI_TR_001  ഞാൻ കുറച്ച് കാലമായി മുച്ചട്ച്ചിൻ്റെ ഫേസ് വാഷ് ...  HUMAN
1  MAL_HUAI_TR_002           ഈ ഫേസ് വാഷ് തണുപ്പ് വെതറിലും ഉപയോഗിക്കാം  HUMAN
2  MAL_HUAI_TR_003  അണ്ണാ എനിക്ക് 14 വയസ് ആയ തേയോളു എനിക്ക് സ്കിൻക...  HUMAN
3  MAL_HUAI_TR_004  ബ്രോ ഇതെല്ലം യൂസ്  ആക്കീട്ട് നൈറ്റ് പിന്നെ വേറ...  HUMAN
4  MAL_HUAI_TR_005    ഇത് ഫേസ് വാഷ് ഡെയിലി ചെയ്താ സ്കിൻകെയറിന് നല്ലതാ  HUMAN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      800 non-null    object
 1   DATA    800 non-null    object
 2   LABEL   800 non-null    object
dtypes: object(3)
memory usage: 18.9+ KB
None


# PreProcessing and Split Up

In [ ]:
#PreProcessing and Split Up
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split

# Download necessary NLTK resources
nltk.download('punkt')

# Sample Malayalam stop words (You can expand this list as needed)
malayalam_stopwords = [
    'അത്', 'ഇത്', 'അവൻ', 'അവൾ', 'അവ', 'ഞാൻ', 'ഞങ്ങളുടെ', 'ഈ', 'എന്ന', 'ഒരു', 'ഇവൻ', 'ഇവൾ', 'അത്', 'അതിൽ', 'എന്റെ', 'ആ', 'നിങ്ങളുടെ', 'അവിടം'
    # Add more Malayalam stop words here
]

# Function for removing punctuation
def remove_punctuation(text_data):
    # Removing punctuation using regular expression
    punctuation_removed = re.sub(r'[^\w\sഅആഇഈഉഊഋഎഏഐഒഓഔകഖഗഘങചഛജഝഞടഠഡഢണതഥദധനപഫബഭമയരറലളഴവശഷസഹ]', '', text_data)
    return punctuation_removed

# Function for tokenization
def tokenization(text_data):
    # Splitting the sentence into words where space is found
    tokens_text = re.split(' ', text_data)
    return tokens_text

# Function for removing stop words
def remove_stopwords(tokens):
    # Removing stop words
    filtered_tokens = [word for word in tokens if word not in malayalam_stopwords]
    return filtered_tokens

# Function to preprocess the dataset
def preprocess_data(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(file_path)

    # Step 2: Apply preprocessing to the 'DATA' column (replace with actual column name if different)
    # Removing punctuation
    df['cleaned_data'] = df['DATA'].apply(remove_punctuation)

    # Tokenizing the cleaned data
    df['tokenized_data'] = df['cleaned_data'].apply(tokenization)

    # Removing stop words
    df['filtered_data'] = df['tokenized_data'].apply(remove_stopwords)

    # Step 3: Combine tokens back into a single string for each document
    df['processed_text'] = df['filtered_data'].apply(lambda x: ' '.join(x))

    # Step 4: Return the processed data
    return df[['ID', 'DATA', 'LABEL', 'processed_text']]

# Path to the dataset (update this with your actual dataset path)
file_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'  # Adjust the path to your dataset

# Step 5: Preprocess the data
processed_data = preprocess_data(file_path)

# Step 6: Prepare the data for model training
X = processed_data['processed_text']  # Features
y = processed_data['LABEL']  # Labels

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the splits for further processing
print("Data preprocessing and splitting completed.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Data preprocessing and splitting completed.


# Feature Extraction

Logistic Regression with feature extraction


In [ ]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK resources
nltk.download('punkt')

# Sample Malayalam stop words (You can expand this list as needed)
malayalam_stopwords = [
    'ആ', 'ഇ', 'ഈ', 'എ', 'എന്ത്', 'ഞാൻ', 'ഒരു', 'ഇത്', 'അതിന്റെ', 'അവന്റെ', 'അവളുടെ', 'അത്', 'ആരാ', 'എവിടെയാണ്', 'ഉണ്ട്', 'ഉള്ള', 'അല്ല', 'പോലെ'
    # Add more Malayalam stop words here
]

# Function for removing punctuation
def remove_punctuation(text_data):
    # Removing punctuation using regular expression (including Malayalam-specific characters)
    punctuation_removed = re.sub(r'[^\w\sഅആഇഈഉഊഎഏഐഒഓകഖഗഘങചഛജഝഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ]', '', text_data)
    return punctuation_removed

# Function for tokenization
def tokenization(text_data):
    # Splitting the sentence into words where space is found
    tokens_text = re.split(' ', text_data)
    return tokens_text

# Function for removing stop words
def remove_stopwords(tokens):
    # Removing stop words
    filtered_tokens = [word for word in tokens if word not in malayalam_stopwords]
    return filtered_tokens

# Function to preprocess the dataset
def preprocess_data(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(file_path)

    # Step 2: Apply preprocessing to the 'DATA' column (replace with actual column name if different)
    # Removing punctuation
    df['cleaned_data'] = df['DATA'].apply(remove_punctuation)

    # Tokenizing the cleaned data
    df['tokenized_data'] = df['cleaned_data'].apply(tokenization)

    # Removing stop words
    df['filtered_data'] = df['tokenized_data'].apply(remove_stopwords)

    # Step 3: Combine tokens back into a single string for each document
    df['processed_text'] = df['filtered_data'].apply(lambda x: ' '.join(x))

    # Step 4: Return the processed data
    return df[['ID', 'DATA', 'LABEL', 'processed_text']]

# Path to the dataset (update this with your actual dataset path)
file_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'  # Adjust the path to your dataset

# Step 5: Preprocess the data
processed_data = preprocess_data(file_path)

# Step 6: Prepare the data for model training
X = processed_data['processed_text']  # Features
y = processed_data['LABEL']  # Labels

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ------------------------------ 1. Logistic Regression with Bag of Words ------------------------------

# Bag of Words Feature Extraction
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train Logistic Regression model
logreg_bow = LogisticRegression(max_iter=1000)
logreg_bow.fit(X_train_bow, y_train)

# Predict and evaluate
y_pred_bow = logreg_bow.predict(X_test_bow)
print("Bag of Words - Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


# ------------------------------ 2. Logistic Regression with TF-IDF ------------------------------

# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train Logistic Regression model
logreg_tfidf = LogisticRegression(max_iter=1000)
logreg_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_tfidf = logreg_tfidf.predict(X_test_tfidf)
print("TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


# ------------------------------ 3. Logistic Regression with N-gram and TF-IDF ------------------------------

# N-gram and TF-IDF Feature Extraction
ngram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram_tfidf = ngram_tfidf_vectorizer.fit_transform(X_train)
X_test_ngram_tfidf = ngram_tfidf_vectorizer.transform(X_test)

# Train Logistic Regression model
logreg_ngram_tfidf = LogisticRegression(max_iter=1000)
logreg_ngram_tfidf.fit(X_train_ngram_tfidf, y_train)

# Predict and evaluate
y_pred_ngram_tfidf = logreg_ngram_tfidf.predict(X_test_ngram_tfidf)
print("N-gram with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_ngram_tfidf))
print(classification_report(y_test, y_pred_ngram_tfidf))


# ------------------------------ 4. Logistic Regression with Count Vectorization and N-gram ------------------------------

# Count Vectorization with N-gram Feature Extraction
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

# Train Logistic Regression model
logreg_ngram = LogisticRegression(max_iter=1000)
logreg_ngram.fit(X_train_ngram, y_train)

# Predict and evaluate
y_pred_ngram = logreg_ngram.predict(X_test_ngram)
print("Count Vectorization with N-gram - Accuracy:", accuracy_score(y_test, y_pred_ngram))
print(classification_report(y_test, y_pred_ngram))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Bag of Words - Accuracy: 0.7625
              precision    recall  f1-score   support

          AI       0.73      0.84      0.78        80
       HUMAN       0.81      0.69      0.74        80

    accuracy                           0.76       160
   macro avg       0.77      0.76      0.76       160
weighted avg       0.77      0.76      0.76       160

TF-IDF - Accuracy: 0.75625
              precision    recall  f1-score   support

          AI       0.75      0.76      0.76        80
       HUMAN       0.76      0.75      0.75        80

    accuracy                           0.76       160
   macro avg       0.76      0.76      0.76       160
weighted avg       0.76      0.76      0.76       160

N-gram with TF-IDF - Accuracy: 0.76875
              precision    recall  f1-score   support

          AI       0.78      0.75      0.76        80
       HUMAN       0.76      0.79      0.77        80

    accuracy                           0.77       160
   macro avg       0.77      0

SVM with feature extraction

In [ ]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK resources
nltk.download('punkt')

# Sample Malayalam stop words (You can expand this list as needed)
malayalam_stopwords = [
    'ആ', 'ഇ', 'ഈ', 'എ', 'എന്ത്', 'ഞാൻ', 'ഒരു', 'ഇത്', 'അതിന്റെ', 'അവന്റെ', 'അവളുടെ', 'അത്', 'ആരാ', 'എവിടെയാണ്', 'ഉണ്ട്', 'ഉള്ള', 'അല്ല', 'പോലെ'
    # Add more Malayalam stop words here
]

# Function for removing punctuation
def remove_punctuation(text_data):
    # Removing punctuation using regular expression (including Malayalam-specific characters)
    punctuation_removed = re.sub(r'[^\w\sഅആഇഈഉഊഎഏഐഒഓകഖഗഘങചഛജഝഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ]', '', text_data)
    return punctuation_removed

# Function for tokenization
def tokenization(text_data):
    # Splitting the sentence into words where space is found
    tokens_text = re.split(' ', text_data)
    return tokens_text

# Function for removing stop words
def remove_stopwords(tokens):
    # Removing stop words
    filtered_tokens = [word for word in tokens if word not in malayalam_stopwords]
    return filtered_tokens

# Function to preprocess the dataset
def preprocess_data(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(file_path)

    # Step 2: Apply preprocessing to the 'DATA' column (replace with actual column name if different)
    # Removing punctuation
    df['cleaned_data'] = df['DATA'].apply(remove_punctuation)

    # Tokenizing the cleaned data
    df['tokenized_data'] = df['cleaned_data'].apply(tokenization)

    # Removing stop words
    df['filtered_data'] = df['tokenized_data'].apply(remove_stopwords)

    # Step 3: Combine tokens back into a single string for each document
    df['processed_text'] = df['filtered_data'].apply(lambda x: ' '.join(x))

    # Step 4: Return the processed data
    return df[['ID', 'DATA', 'LABEL', 'processed_text']]

# Path to the dataset (update this with your actual dataset path)
file_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'  # Adjust the path to your dataset

# Step 5: Preprocess the data
processed_data = preprocess_data(file_path)

# Step 6: Prepare the data for model training
X = processed_data['processed_text']  # Features
y = processed_data['LABEL']  # Labels

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# ------------------------------ 1. SVM with Bag of Words ------------------------------

# Bag of Words Feature Extraction
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train SVM model
svm_bow = SVC(kernel='linear')
svm_bow.fit(X_train_bow, y_train)

# Predict and evaluate
y_pred_bow = svm_bow.predict(X_test_bow)
print("SVM with Bag of Words - Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


# ------------------------------ 2. SVM with TF-IDF ------------------------------

# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train SVM model
svm_tfidf = SVC(kernel='linear')
svm_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_tfidf = svm_tfidf.predict(X_test_tfidf)
print("SVM with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


# ------------------------------ 3. SVM with N-gram and TF-IDF ------------------------------

# N-gram and TF-IDF Feature Extraction
ngram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram_tfidf = ngram_tfidf_vectorizer.fit_transform(X_train)
X_test_ngram_tfidf = ngram_tfidf_vectorizer.transform(X_test)

# Train SVM model
svm_ngram_tfidf = SVC(kernel='linear')
svm_ngram_tfidf.fit(X_train_ngram_tfidf, y_train)

# Predict and evaluate
y_pred_ngram_tfidf = svm_ngram_tfidf.predict(X_test_ngram_tfidf)
print("SVM with N-gram and TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_ngram_tfidf))
print(classification_report(y_test, y_pred_ngram_tfidf))


# ------------------------------ 4. SVM with Count Vectorization and N-gram ------------------------------

# Count Vectorization with N-gram Feature Extraction
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

# Train SVM model
svm_ngram = SVC(kernel='linear')
svm_ngram.fit(X_train_ngram, y_train)

# Predict and evaluate
y_pred_ngram = svm_ngram.predict(X_test_ngram)
print("SVM with Count Vectorization and N-gram - Accuracy:", accuracy_score(y_test, y_pred_ngram))
print(classification_report(y_test, y_pred_ngram))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


SVM with Bag of Words - Accuracy: 0.69375
              precision    recall  f1-score   support

          AI       0.68      0.74      0.71        80
       HUMAN       0.71      0.65      0.68        80

    accuracy                           0.69       160
   macro avg       0.70      0.69      0.69       160
weighted avg       0.70      0.69      0.69       160

SVM with TF-IDF - Accuracy: 0.725
              precision    recall  f1-score   support

          AI       0.72      0.74      0.73        80
       HUMAN       0.73      0.71      0.72        80

    accuracy                           0.72       160
   macro avg       0.73      0.73      0.72       160
weighted avg       0.73      0.72      0.72       160

SVM with N-gram and TF-IDF - Accuracy: 0.75625
              precision    recall  f1-score   support

          AI       0.76      0.75      0.75        80
       HUMAN       0.75      0.76      0.76        80

    accuracy                           0.76       160
   ma

Random Forest with feature extraction

In [ ]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK resources
nltk.download('punkt')

# Sample Malayalam stop words (You can expand this list as needed)
malayalam_stopwords = [
    'ആ', 'ഇ', 'ഈ', 'എ', 'എന്ത്', 'ഞാൻ', 'ഒരു', 'ഇത്', 'അതിന്റെ', 'അവന്റെ', 'അവളുടെ', 'അത്', 'ആരാ', 'എവിടെയാണ്', 'ഉണ്ട്', 'ഉള്ള', 'അല്ല', 'പോലെ'
    # Add more Malayalam stop words here
]

# Function for removing punctuation
def remove_punctuation(text_data):
    # Removing punctuation using regular expression (including Malayalam-specific characters)
    punctuation_removed = re.sub(r'[^\w\sഅആഇഈഉഊഎഏഐഒഓകഖഗഘങചഛജഝഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ]', '', text_data)
    return punctuation_removed

# Function for tokenization
def tokenization(text_data):
    # Splitting the sentence into words where space is found
    tokens_text = re.split(' ', text_data)
    return tokens_text

# Function for removing stop words
def remove_stopwords(tokens):
    # Removing stop words
    filtered_tokens = [word for word in tokens if word not in malayalam_stopwords]
    return filtered_tokens

# Function to preprocess the dataset
def preprocess_data(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(file_path)

    # Step 2: Apply preprocessing to the 'DATA' column (replace with actual column name if different)
    # Removing punctuation
    df['cleaned_data'] = df['DATA'].apply(remove_punctuation)

    # Tokenizing the cleaned data
    df['tokenized_data'] = df['cleaned_data'].apply(tokenization)

    # Removing stop words
    df['filtered_data'] = df['tokenized_data'].apply(remove_stopwords)

    # Step 3: Combine tokens back into a single string for each document
    df['processed_text'] = df['filtered_data'].apply(lambda x: ' '.join(x))

    # Step 4: Return the processed data
    return df[['ID', 'DATA', 'LABEL', 'processed_text']]

# Path to the dataset (update this with your actual dataset path)
file_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'  # Adjust the path to your dataset

# Step 5: Preprocess the data
processed_data = preprocess_data(file_path)

# Step 6: Prepare the data for model training
X = processed_data['processed_text']  # Features
y = processed_data['LABEL']  # Labels

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# ------------------------------ 1. Random Forest with Bag of Words ------------------------------

# Bag of Words Feature Extraction
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train Random Forest model
rf_bow = RandomForestClassifier(n_estimators=100, random_state=42)
rf_bow.fit(X_train_bow, y_train)

# Predict and evaluate
y_pred_bow = rf_bow.predict(X_test_bow)
print("Random Forest with Bag of Words - Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


# ------------------------------ 2. Random Forest with TF-IDF ------------------------------

# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train Random Forest model
rf_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_tfidf = rf_tfidf.predict(X_test_tfidf)
print("Random Forest with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


# ------------------------------ 3. Random Forest with N-gram and TF-IDF ------------------------------

# N-gram and TF-IDF Feature Extraction
ngram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram_tfidf = ngram_tfidf_vectorizer.fit_transform(X_train)
X_test_ngram_tfidf = ngram_tfidf_vectorizer.transform(X_test)

# Train Random Forest model
rf_ngram_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_ngram_tfidf.fit(X_train_ngram_tfidf, y_train)

# Predict and evaluate
y_pred_ngram_tfidf = rf_ngram_tfidf.predict(X_test_ngram_tfidf)
print("Random Forest with N-gram and TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_ngram_tfidf))
print(classification_report(y_test, y_pred_ngram_tfidf))


# ------------------------------ 4. Random Forest with Count Vectorization and N-gram ------------------------------

# Count Vectorization with N-gram Feature Extraction
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

# Train Random Forest model
rf_ngram = RandomForestClassifier(n_estimators=100, random_state=42)
rf_ngram.fit(X_train_ngram, y_train)

# Predict and evaluate
y_pred_ngram = rf_ngram.predict(X_test_ngram)
print("Random Forest with Count Vectorization and N-gram - Accuracy:", accuracy_score(y_test, y_pred_ngram))
print(classification_report(y_test, y_pred_ngram))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Random Forest with Bag of Words - Accuracy: 0.75625
              precision    recall  f1-score   support

          AI       0.70      0.91      0.79        80
       HUMAN       0.87      0.60      0.71        80

    accuracy                           0.76       160
   macro avg       0.78      0.76      0.75       160
weighted avg       0.78      0.76      0.75       160

Random Forest with TF-IDF - Accuracy: 0.7875
              precision    recall  f1-score   support

          AI       0.77      0.82      0.80        80
       HUMAN       0.81      0.75      0.78        80

    accuracy                           0.79       160
   macro avg       0.79      0.79      0.79       160
weighted avg       0.79      0.79      0.79       160

Random Forest with N-gram and TF-IDF - Accuracy: 0.75
              precision    recall  f1-score   support

          AI       0.79      0.69      0.73        80
       HUMAN       0.72      0.81      0.76        80

    accuracy                   

Prediction

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Path to the dataset
file_path = '/content/drive/Shareddrives/AI Text Detecting/mal_test_data_hum_ai.xlsx'  # Dataset path
output_tsv_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_test_data_with_RFgenerated_labels.tsv'

# Step 1: Load the dataset
try:
    df = pd.read_excel(file_path)  # Load Excel file
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')  # Fallback for non-UTF-8 encoded CSV files

# Debugging: Print column names
print("Columns in dataset:", df.columns)

# Step 2: Process the dataset
if 'DATA' in df.columns:
    X = df['DATA']  # Use 'DATA' for features
    # Assign "Human" and "AI" labels as an example (replace with actual label logic if available)
    y = ['Human' if i % 2 == 0 else 'AI' for i in range(len(X))]  # Example: Alternate between Human and AI
else:
    raise ValueError("The dataset must contain a 'DATA' column for text data.")

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train.astype(str))  # Convert to string
X_test_tfidf = tfidf_vectorizer.transform(X_test.astype(str))

# Step 5: Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Step 6: Evaluate the model
y_pred = rf_model.predict(X_test_tfidf)
print("Random Forest with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Step 7: Predict labels for the entire dataset
all_data_tfidf = tfidf_vectorizer.transform(X.astype(str))
predicted_labels = rf_model.predict(all_data_tfidf)

# Step 8: Add predicted labels to the dataset
df['PREDICTED_LABEL'] = predicted_labels

# Step 9: Save the results to a TSV file
df.to_csv(output_tsv_path, sep='\t', index=False)

print(f"Predictions saved to {output_tsv_path}")
print(df.head())


Columns in dataset: Index(['ID', 'DATA'], dtype='object')
Random Forest with TF-IDF - Accuracy: 0.45
              precision    recall  f1-score   support

          AI       0.39      0.69      0.50        16
       Human       0.58      0.29      0.39        24

    accuracy                           0.45        40
   macro avg       0.49      0.49      0.44        40
weighted avg       0.51      0.45      0.43        40

Predictions saved to /content/drive/Shareddrives/AI Text Detecting/malayalam_test_data_with_RFgenerated_labels.tsv
                ID                                               DATA  \
0  MAL_HUAI_TE_001  എല്ലാം നന്നായിട്ടുണ്ട് പക്ഷെ എനിക്ക് മീനിന്റെ ...   
1  MAL_HUAI_TE_002  ബിലാൽ തട്ടുകടയിൽ നിന്നും പഴകിയ ഫുഡ് കുറെ പ്രാവ...   
2  MAL_HUAI_TE_003  കോവയ്ക്ക ഉപ്പിലിട്ടത് എൻറെ ജീവിതത്തിൽ ഇതുവരെ ക...   
3  MAL_HUAI_TE_004  കേരളത്തിൽ ഏറ്റവും നല്ല ഭക്ഷണം കിട്ടുന്നത് കോഴി...   
4  MAL_HUAI_TE_005  എല്ലാം അടിപൊളി, പക്ഷെ സുരേന്ദ്രൻ തട്ടുകടയിൽ തി...   

  PREDICTED_LABEL 

KNN

In [ ]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK resources
nltk.download('punkt')

# Sample Malayalam stop words (You can expand this list as needed)
malayalam_stopwords = [
    'ആ', 'ഇ', 'ഈ', 'എ', 'എന്ത്', 'ഞാൻ', 'ഒരു', 'ഇത്', 'അതിന്റെ', 'അവന്റെ', 'അവളുടെ', 'അത്', 'ആരാ', 'എവിടെയാണ്', 'ഉണ്ട്', 'ഉള്ള', 'അല്ല', 'പോലെ'
    # Add more Malayalam stop words here
]

# Function for removing punctuation
def remove_punctuation(text_data):
    # Removing punctuation using regular expression (including Malayalam-specific characters)
    punctuation_removed = re.sub(r'[^\w\sഅആഇഈഉഊഎഏഐഒഓകഖഗഘങചഛജഝഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ]', '', text_data)
    return punctuation_removed

# Function for tokenization
def tokenization(text_data):
    # Splitting the sentence into words where space is found
    tokens_text = re.split(' ', text_data)
    return tokens_text

# Function for removing stop words
def remove_stopwords(tokens):
    # Removing stop words
    filtered_tokens = [word for word in tokens if word not in malayalam_stopwords]
    return filtered_tokens

# Function to preprocess the dataset
def preprocess_data(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(file_path)

    # Step 2: Apply preprocessing to the 'DATA' column (replace with actual column name if different)
    # Removing punctuation
    df['cleaned_data'] = df['DATA'].apply(remove_punctuation)

    # Tokenizing the cleaned data
    df['tokenized_data'] = df['cleaned_data'].apply(tokenization)

    # Removing stop words
    df['filtered_data'] = df['tokenized_data'].apply(remove_stopwords)

    # Step 3: Combine tokens back into a single string for each document
    df['processed_text'] = df['filtered_data'].apply(lambda x: ' '.join(x))

    # Step 4: Return the processed data
    return df[['ID', 'DATA', 'LABEL', 'processed_text']]

# Path to the dataset (update this with your actual dataset path)
file_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'  # Adjust the path to your dataset

# Step 5: Preprocess the data
processed_data = preprocess_data(file_path)

# Step 6: Prepare the data for model training
X = processed_data['processed_text']  # Features
y = processed_data['LABEL']  # Labels

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# ------------------------------ 1. KNN with Bag of Words ------------------------------

# Bag of Words Feature Extraction
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train KNN model
knn_bow = KNeighborsClassifier(n_neighbors=5)
knn_bow.fit(X_train_bow, y_train)

# Predict and evaluate
y_pred_bow = knn_bow.predict(X_test_bow)
print("KNN with Bag of Words - Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


# ------------------------------ 2. KNN with TF-IDF ------------------------------

# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train KNN model
knn_tfidf = KNeighborsClassifier(n_neighbors=5)
knn_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_tfidf = knn_tfidf.predict(X_test_tfidf)
print("KNN with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


# ------------------------------ 3. KNN with N-gram and TF-IDF ------------------------------

# N-gram and TF-IDF Feature Extraction
ngram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram_tfidf = ngram_tfidf_vectorizer.fit_transform(X_train)
X_test_ngram_tfidf = ngram_tfidf_vectorizer.transform(X_test)

# Train KNN model
knn_ngram_tfidf = KNeighborsClassifier(n_neighbors=5)
knn_ngram_tfidf.fit(X_train_ngram_tfidf, y_train)

# Predict and evaluate
y_pred_ngram_tfidf = knn_ngram_tfidf.predict(X_test_ngram_tfidf)
print("KNN with N-gram and TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_ngram_tfidf))
print(classification_report(y_test, y_pred_ngram_tfidf))


# ------------------------------ 4. KNN with Count Vectorization and N-gram ------------------------------

# Count Vectorization with N-gram Feature Extraction
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

# Train KNN model
knn_ngram = KNeighborsClassifier(n_neighbors=5)
knn_ngram.fit(X_train_ngram, y_train)

# Predict and evaluate
y_pred_ngram = knn_ngram.predict(X_test_ngram)
print("KNN with Count Vectorization and N-gram - Accuracy:", accuracy_score(y_test, y_pred_ngram))
print(classification_report(y_test, y_pred_ngram))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KNN with Bag of Words - Accuracy: 0.51875
              precision    recall  f1-score   support

          AI       0.67      0.07      0.13        80
       HUMAN       0.51      0.96      0.67        80

    accuracy                           0.52       160
   macro avg       0.59      0.52      0.40       160
weighted avg       0.59      0.52      0.40       160

KNN with TF-IDF - Accuracy: 0.73125
              precision    recall  f1-score   support

          AI       0.72      0.76      0.74        80
       HUMAN       0.75      0.70      0.72        80

    accuracy                           0.73       160
   macro avg       0.73      0.73      0.73       160
weighted avg       0.73      0.73      0.73       160

KNN with N-gram and TF-IDF - Accuracy: 0.7125
              precision    recall  f1-score   support

          AI       0.71      0.71      0.71        80
       HUMAN       0.71      0.71      0.71        80

    accuracy                           0.71       160
   m

Decision Tree with feature extraction

In [ ]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK resources
nltk.download('punkt')

# Sample Malayalam stop words (You can expand this list as needed)
malayalam_stopwords = [
    'ആ', 'ഇ', 'ഈ', 'എ', 'എന്ത്', 'ഞാൻ', 'ഒരു', 'ഇത്', 'അതിന്റെ', 'അവന്റെ', 'അവളുടെ', 'അത്', 'ആരാ', 'എവിടെയാണ്', 'ഉണ്ട്', 'ഉള്ള', 'അല്ല', 'പോലെ'
    # Add more Malayalam stop words here
]

# Function for removing punctuation
def remove_punctuation(text_data):
    # Removing punctuation using regular expression (including Malayalam-specific characters)
    punctuation_removed = re.sub(r'[^\w\sഅആഇഈഉഊഎഏഐഒഓകഖഗഘങചഛജഝഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ]', '', text_data)
    return punctuation_removed

# Function for tokenization
def tokenization(text_data):
    # Splitting the sentence into words where space is found
    tokens_text = re.split(' ', text_data)
    return tokens_text

# Function for removing stop words
def remove_stopwords(tokens):
    # Removing stop words
    filtered_tokens = [word for word in tokens if word not in malayalam_stopwords]
    return filtered_tokens

# Function to preprocess the dataset
def preprocess_data(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(file_path)

    # Step 2: Apply preprocessing to the 'DATA' column (replace with actual column name if different)
    # Removing punctuation
    df['cleaned_data'] = df['DATA'].apply(remove_punctuation)

    # Tokenizing the cleaned data
    df['tokenized_data'] = df['cleaned_data'].apply(tokenization)

    # Removing stop words
    df['filtered_data'] = df['tokenized_data'].apply(remove_stopwords)

    # Step 3: Combine tokens back into a single string for each document
    df['processed_text'] = df['filtered_data'].apply(lambda x: ' '.join(x))

    # Step 4: Return the processed data
    return df[['ID', 'DATA', 'LABEL', 'processed_text']]

# Path to the dataset (update this with your actual dataset path)
file_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'  # Adjust the path to your dataset

# Step 5: Preprocess the data
processed_data = preprocess_data(file_path)

# Step 6: Prepare the data for model training
X = processed_data['processed_text']  # Features
y = processed_data['LABEL']  # Labels

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# ------------------------------ 1. Decision Tree with Bag of Words ------------------------------

# Bag of Words Feature Extraction
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train Decision Tree model
dt_bow = DecisionTreeClassifier(random_state=42)
dt_bow.fit(X_train_bow, y_train)

# Predict and evaluate
y_pred_bow = dt_bow.predict(X_test_bow)
print("Decision Tree with Bag of Words - Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


# ------------------------------ 2. Decision Tree with TF-IDF ------------------------------

# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train Decision Tree model
dt_tfidf = DecisionTreeClassifier(random_state=42)
dt_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_tfidf = dt_tfidf.predict(X_test_tfidf)
print("Decision Tree with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


# ------------------------------ 3. Decision Tree with N-gram and TF-IDF ------------------------------

# N-gram and TF-IDF Feature Extraction
ngram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram_tfidf = ngram_tfidf_vectorizer.fit_transform(X_train)
X_test_ngram_tfidf = ngram_tfidf_vectorizer.transform(X_test)

# Train Decision Tree model
dt_ngram_tfidf = DecisionTreeClassifier(random_state=42)
dt_ngram_tfidf.fit(X_train_ngram_tfidf, y_train)

# Predict and evaluate
y_pred_ngram_tfidf = dt_ngram_tfidf.predict(X_test_ngram_tfidf)
print("Decision Tree with N-gram and TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_ngram_tfidf))
print(classification_report(y_test, y_pred_ngram_tfidf))


# ------------------------------ 4. Decision Tree with Count Vectorization and N-gram ------------------------------

# Count Vectorization with N-gram Feature Extraction
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

# Train Decision Tree model
dt_ngram = DecisionTreeClassifier(random_state=42)
dt_ngram.fit(X_train_ngram, y_train)

# Predict and evaluate
y_pred_ngram = dt_ngram.predict(X_test_ngram)
print("Decision Tree with Count Vectorization and N-gram - Accuracy:", accuracy_score(y_test, y_pred_ngram))
print(classification_report(y_test, y_pred_ngram))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Decision Tree with Bag of Words - Accuracy: 0.73125
              precision    recall  f1-score   support

          AI       0.69      0.84      0.76        80
       HUMAN       0.79      0.62      0.70        80

    accuracy                           0.73       160
   macro avg       0.74      0.73      0.73       160
weighted avg       0.74      0.73      0.73       160

Decision Tree with TF-IDF - Accuracy: 0.725
              precision    recall  f1-score   support

          AI       0.71      0.75      0.73        80
       HUMAN       0.74      0.70      0.72        80

    accuracy                           0.72       160
   macro avg       0.73      0.72      0.72       160
weighted avg       0.73      0.72      0.72       160

Decision Tree with N-gram and TF-IDF - Accuracy: 0.69375
              precision    recall  f1-score   support

          AI       0.70      0.69      0.69        80
       HUMAN       0.69      0.70      0.70        80

    accuracy                 

Naive Bayes with feature extraction

In [ ]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK resources
nltk.download('punkt')

# Sample Malayalam stop words (You can expand this list as needed)
malayalam_stopwords = [
    'ആ', 'ഇ', 'ഈ', 'എ', 'എന്ത്', 'ഞാൻ', 'ഒരു', 'ഇത്', 'അതിന്റെ', 'അവന്റെ', 'അവളുടെ', 'അത്', 'ആരാ', 'എവിടെയാണ്', 'ഉണ്ട്', 'ഉള്ള', 'അല്ല', 'പോലെ'
    # Add more Malayalam stop words here
]

# Function for removing punctuation
def remove_punctuation(text_data):
    # Removing punctuation using regular expression (including Malayalam-specific characters)
    punctuation_removed = re.sub(r'[^\w\sഅആഇഈഉഊഎഏഐഒഓകഖഗഘങചഛജഝഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ]', '', text_data)
    return punctuation_removed

# Function for tokenization
def tokenization(text_data):
    # Splitting the sentence into words where space is found
    tokens_text = re.split(' ', text_data)
    return tokens_text

# Function for removing stop words
def remove_stopwords(tokens):
    # Removing stop words
    filtered_tokens = [word for word in tokens if word not in malayalam_stopwords]
    return filtered_tokens

# Function to preprocess the dataset
def preprocess_data(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(file_path)

    # Step 2: Apply preprocessing to the 'DATA' column (replace with actual column name if different)
    # Removing punctuation
    df['cleaned_data'] = df['DATA'].apply(remove_punctuation)

    # Tokenizing the cleaned data
    df['tokenized_data'] = df['cleaned_data'].apply(tokenization)

    # Removing stop words
    df['filtered_data'] = df['tokenized_data'].apply(remove_stopwords)

    # Step 3: Combine tokens back into a single string for each document
    df['processed_text'] = df['filtered_data'].apply(lambda x: ' '.join(x))

    # Step 4: Return the processed data
    return df[['ID', 'DATA', 'LABEL', 'processed_text']]

# Path to the dataset (update this with your actual dataset path)
file_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv'  # Adjust the path to your dataset

# Step 5: Preprocess the data
processed_data = preprocess_data(file_path)

# Step 6: Prepare the data for model training
X = processed_data['processed_text']  # Features
y = processed_data['LABEL']  # Labels

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# ------------------------------ 1. Naive Bayes with Bag of Words ------------------------------

# Bag of Words Feature Extraction
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train Naive Bayes model
nb_bow = MultinomialNB()
nb_bow.fit(X_train_bow, y_train)

# Predict and evaluate
y_pred_bow = nb_bow.predict(X_test_bow)
print("Naive Bayes with Bag of Words - Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


# ------------------------------ 2. Naive Bayes with TF-IDF ------------------------------

# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train Naive Bayes model
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)
print("Naive Bayes with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


# ------------------------------ 3. Naive Bayes with N-gram and TF-IDF ------------------------------

# N-gram and TF-IDF Feature Extraction
ngram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram_tfidf = ngram_tfidf_vectorizer.fit_transform(X_train)
X_test_ngram_tfidf = ngram_tfidf_vectorizer.transform(X_test)

# Train Naive Bayes model
nb_ngram_tfidf = MultinomialNB()
nb_ngram_tfidf.fit(X_train_ngram_tfidf, y_train)

# Predict and evaluate
y_pred_ngram_tfidf = nb_ngram_tfidf.predict(X_test_ngram_tfidf)
print("Naive Bayes with N-gram and TF-IDF - Accuracy:", accuracy_score(y_test, y_pred_ngram_tfidf))
print(classification_report(y_test, y_pred_ngram_tfidf))


# ------------------------------ 4. Naive Bayes with Count Vectorization and N-gram ------------------------------

# Count Vectorization with N-gram Feature Extraction
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using unigrams and bigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

# Train Naive Bayes model
nb_ngram = MultinomialNB()
nb_ngram.fit(X_train_ngram, y_train)

# Predict and evaluate
y_pred_ngram = nb_ngram.predict(X_test_ngram)
print("Naive Bayes with Count Vectorization and N-gram - Accuracy:", accuracy_score(y_test, y_pred_ngram))
print(classification_report(y_test, y_pred_ngram))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Naive Bayes with Bag of Words - Accuracy: 0.7625
              precision    recall  f1-score   support

          AI       0.76      0.76      0.76        80
       HUMAN       0.76      0.76      0.76        80

    accuracy                           0.76       160
   macro avg       0.76      0.76      0.76       160
weighted avg       0.76      0.76      0.76       160

Naive Bayes with TF-IDF - Accuracy: 0.75
              precision    recall  f1-score   support

          AI       0.74      0.76      0.75        80
       HUMAN       0.76      0.74      0.75        80

    accuracy                           0.75       160
   macro avg       0.75      0.75      0.75       160
weighted avg       0.75      0.75      0.75       160

Naive Bayes with N-gram and TF-IDF - Accuracy: 0.7625
              precision    recall  f1-score   support

          AI       0.76      0.78      0.77        80
       HUMAN       0.77      0.75      0.76        80

    accuracy                          

Prediction

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Path to the dataset
file_path = '/content/drive/Shareddrives/AI Text Detecting/mal_test_data_hum_ai.xlsx'  # Dataset path
output_tsv_path = '/content/drive/Shareddrives/AI Text Detecting/malayalam_test_data_with_Ngenerated_labels.tsv'

# Step 1: Load the dataset
try:
    df = pd.read_excel(file_path)  # Load Excel file
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')  # Fallback for non-UTF-8 encoded CSV files

# Debugging: Print column names
print("Columns in dataset:", df.columns)

# Step 2: Process the dataset
if 'DATA' in df.columns:
    X = df['DATA']  # Use 'DATA' for features
    # Assign "Human" and "AI" labels as an example (replace with actual label logic if available)
    y = ['Human' if i % 2 == 0 else 'AI' for i in range(len(X))]  # Example: Alternate between Human and AI
else:
    raise ValueError("The dataset must contain a 'DATA' column for text data.")

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train.astype(str))  # Convert to string
X_test_tfidf = tfidf_vectorizer.transform(X_test.astype(str))

# Step 5: Train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Step 6: Evaluate the model
y_pred = nb_model.predict(X_test_tfidf)
print("Naive Bayes with TF-IDF - Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Step 7: Predict labels for the entire dataset
all_data_tfidf = tfidf_vectorizer.transform(X.astype(str))
predicted_labels = nb_model.predict(all_data_tfidf)

# Step 8: Add predicted labels to the dataset
df['PREDICTED_LABEL'] = predicted_labels

# Step 9: Save the results to a TSV file
df.to_csv(output_tsv_path, sep='\t', index=False)

print(f"Predictions saved to {output_tsv_path}")
print(df.head())


Columns in dataset: Index(['ID', 'DATA'], dtype='object')
Naive Bayes with TF-IDF - Accuracy: 0.5
              precision    recall  f1-score   support

          AI       0.43      0.75      0.55        16
       Human       0.67      0.33      0.44        24

    accuracy                           0.50        40
   macro avg       0.55      0.54      0.49        40
weighted avg       0.57      0.50      0.48        40

Predictions saved to /content/drive/Shareddrives/AI Text Detecting/malayalam_test_data_with_Ngenerated_labels.tsv
                ID                                               DATA  \
0  MAL_HUAI_TE_001  എല്ലാം നന്നായിട്ടുണ്ട് പക്ഷെ എനിക്ക് മീനിന്റെ ...   
1  MAL_HUAI_TE_002  ബിലാൽ തട്ടുകടയിൽ നിന്നും പഴകിയ ഫുഡ് കുറെ പ്രാവ...   
2  MAL_HUAI_TE_003  കോവയ്ക്ക ഉപ്പിലിട്ടത് എൻറെ ജീവിതത്തിൽ ഇതുവരെ ക...   
3  MAL_HUAI_TE_004  കേരളത്തിൽ ഏറ്റവും നല്ല ഭക്ഷണം കിട്ടുന്നത് കോഴി...   
4  MAL_HUAI_TE_005  എല്ലാം അടിപൊളി, പക്ഷെ സുരേന്ദ്രൻ തട്ടുകടയിൽ തി...   

  PREDICTED_LABEL  
0 

# Bert Model

In [ ]:
pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install datasets

In [ ]:
!pip install torch

In [ ]:
!pip install transformers


In [ ]:
pip install --upgrade transformers tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 877.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.1 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Load the dataset
data = pd.read_csv("/content/drive/Shareddrives/AI Text Detecting/malayalam_dataset.csv")  # Replace with your dataset's filename
reviews = data["DATA"]  # Replace 'DATA' with your column name for reviews
labels = data["LABEL"]  # Replace 'LABEL' with your column name for labels

# Sample Malayalam stop words (expand this list as needed)
malayalam_stopwords = [
    'അത', 'ഇത്', 'അവന്', 'അവള്', 'എന്റെ', 'ഈ', 'എന്ന്', 'ഒരു', 'ഇല്ല', 'ഉള്ള',
    'ഇല്', 'എനിക്കു', 'എന്റെ', 'അവൻ', 'അവളുടെ', 'നീ', 'ഇവിടെ'
]

# Preprocess function
def preprocess_text(text, stop_words):
    # Tokenize and remove stop words
    words = text.split()
    cleaned_words = [word for word in words if word not in stop_words]
    return " ".join(cleaned_words)

# Preprocess the reviews
data["cleaned_review"] = data["DATA"].apply(lambda x: preprocess_text(x, malayalam_stopwords))

# Encode labels to integers if not already done
label_mapping = {label: idx for idx, label in enumerate(data["LABEL"].unique())}
data["encoded_label"] = data["LABEL"].map(label_mapping)

# Split into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["cleaned_review"], data["encoded_label"], test_size=0.2, random_state=42
)

# Tokenize data using a transformer tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Create custom dataset class
class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = ReviewsDataset(train_encodings, list(train_labels))
test_dataset = ReviewsDataset(test_encodings, list(test_labels))

# Load transformer model
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", num_labels=len(label_mapping)
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    report_to = "none"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./malayalam_review_model")
tokenizer.save_pretrained("./malayalam_review_model")

# Optional: Evaluate the model
results = trainer.evaluate()
print("Evaluation results:", results)

# Test the model with new data (optional)
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probs).item()
    return {v: k for k, v in label_mapping.items()}[predicted_class]

# Example prediction
example_review = "ഞാൻ കുറച്ച് കാലമായി മുച്ചട്ച്ചിൻ്റെ ഫേസ് വാഷ് ഉഭയോഗിക്കുന്നുണ്ട് നല്ല പ്രോഡക്റ്റ് ആണ്."
print("Predicted label:", predict(example_review))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.222600,0.198284
2,0.140400,0.169896
3,0.100900,0.178555


Evaluation results: {'eval_loss': 0.178555428981781, 'eval_runtime': 70.591, 'eval_samples_per_second': 2.267, 'eval_steps_per_second': 0.142, 'epoch': 3.0}
Predicted label: HUMAN


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions and true labels
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Generate classification report
true_labels = test_labels.tolist()
class_report = classification_report(
    true_labels,
    predicted_labels,
    target_names=[label for label in label_mapping.keys()],
    zero_division=0
)

print("Classification Report:")
print(class_report)


Classification Report:
              precision    recall  f1-score   support

       HUMAN       1.00      0.91      0.95        80
          AI       0.92      1.00      0.96        80

    accuracy                           0.96       160
   macro avg       0.96      0.96      0.96       160
weighted avg       0.96      0.96      0.96       160



Prediction

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Paths to files and model
test_data_path = "/content/drive/Shareddrives/AI Text Detecting/mal_test_data_hum_ai.xlsx"  # Replace with your test data path
model_path = "./malayalam_review_model"  # Path to your trained Malayalam model
output_tsv_path = "/content/drive/Shareddrives/AI Text Detecting/malayalam_test_data_with_generated_labels.tsv"  # Path for the output TSV file

# Load test data from .xlsx file
test_data = pd.read_excel(test_data_path)  # Adjust if your file has multiple sheets
test_texts = test_data["DATA"]

# Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Function to make predictions
def predict(texts):
    predictions = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probs).item()
        predictions.append(predicted_class)
    return predictions

# Generate predictions
predicted_labels = predict(test_texts)

# Map predictions to AI or Human (modify as per your trained labels)
label_mapping = {0: "Human", 1: "AI"}  # Adjust based on your trained labels

# Create the LABEL column with predictions
test_data["LABEL"] = [label_mapping[label] for label in predicted_labels]

# Save the results to a TSV file
test_data.to_csv(output_tsv_path, sep="\t", index=False)

print(f"Predictions saved to {output_tsv_path}")

# Print out the first few rows of the resulting DataFrame
print(test_data.head())


Predictions saved to /content/drive/Shareddrives/AI Text Detecting/malayalam_test_data_with_generated_labels.tsv
                ID                                               DATA  LABEL
0  MAL_HUAI_TE_001  എല്ലാം നന്നായിട്ടുണ്ട് പക്ഷെ എനിക്ക് മീനിന്റെ ...  Human
1  MAL_HUAI_TE_002  ബിലാൽ തട്ടുകടയിൽ നിന്നും പഴകിയ ഫുഡ് കുറെ പ്രാവ...  Human
2  MAL_HUAI_TE_003  കോവയ്ക്ക ഉപ്പിലിട്ടത് എൻറെ ജീവിതത്തിൽ ഇതുവരെ ക...  Human
3  MAL_HUAI_TE_004  കേരളത്തിൽ ഏറ്റവും നല്ല ഭക്ഷണം കിട്ടുന്നത് കോഴി...  Human
4  MAL_HUAI_TE_005  എല്ലാം അടിപൊളി, പക്ഷെ സുരേന്ദ്രൻ തട്ടുകടയിൽ തി...  Human
